In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del__1_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

504586

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

3746926

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1700474299
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 25229235


b'chr22\t1546\tG\tK\t+\nchr22\t1844\t-\tT\t+\nchr22\t2312\tC\tA\t-\nchr22\t2987\tA\tM\t+\nchr22\t3300\tC\tG\t-\nchr22\t3801\tC\tM\t+\nchr22\t5295\tA\tM\t+\nchr22\t5491\tC\tY\t+\nchr22\t6123\tT\tK\t+\nchr22\t6596\tT\tA\t-\nchr22\t7749\tG\tR\t+\nchr22\t9387\tC\tA\t-\nchr22\t9580\tA\tM\t+\nchr22\t9715\t-\tC\t+\nchr22\t10071\tC\tM\t+\nchr22\t10297\tC\tY\t+\nchr22\t11320\tC\tM\t+\nchr22\t11821\tG\tR\t+\nchr22\t13629\tG\tK\t+\nchr22\t13913\tG\tC\t-\nchr22\t14087\tG\tA\t-\nchr22\t14118\tA\tW\t+\nchr22\t14606\tC\tS\t+\nchr22\t14712\t-\tCC\t-\nchr22\t14800\tA\tT\t-\nchr22\t14805\tT\tW\t+\nchr22\t15283\tC\tG\t-\nchr22\t18470\tA\tG\t-\nchr22\t19785\tA\tR\t+\nchr22\t20538\tA\tR\t+\nchr22\t20726\tG\tS\t+\nchr22\t22203\tA\tM\t+\nchr22\t23150\tT\tW\t+\nchr22\t23476\tT\tW\t+\nchr22\t25325\tA\tC\t-\nchr22\t25345\tG\tC\t-\nchr22\t25584\tT\t-\t-\nchr22\t26781\tGA\t-\t-\nchr22\t27728\t-\tC\t+\nchr22\t27794\tT\tA\t-\nchr22\t27864\tC\tM\t+\nchr22\t28932\tT\tY\t+\nchr22\t30706\tC\tG\t-\nchr22\t30754\t-\tC\t+

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 4 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4763	8.800949e-06	7.030950e-66	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.3885	0.000000e+00	2.947080e-283	0.000000e+00	1.603387e-252	0.0000	1.0000	0
duplication	chr22:2926201-2963000	36800	3.1129	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4546401-4713300	166900	1.6554	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.4135	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854500	6200	1.6659	0.000000e+00	6.075626e-269	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047700	8300	1.9436	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9423901-9461600	37700	1.4757	0.000000e+00	8.829544e-81	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13230601-13395900	165300	1.3985	0.000000e+00	0.000000e+00	0.0000

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4763018069305216,0.0
duplication,chr22,1306101,1325100,19000,1.3884981985960942,0.0
duplication,chr22,2926201,2963000,36800,3.112864607994232,0.0
duplication,chr22,4546401,4713300,166900,1.6554067070809841,0.0
duplication,chr22,6655001,6661800,6800,2.4134523371823406,0.0
duplication,chr22,6848301,6854500,6200,1.665931641533973,0.0
duplication,chr22,8039401,8047700,8300,1.9436063295292572,0.0
duplication,chr22,9423901,9461600,37700,1.4757443288543735,0.0
duplication,chr22,13230601,13395900,165300,1.3985092835797683,0.0
duplication,chr22,13481101,13512800,31700,4.21835107279668,0.0
duplication,chr22,15242601,15246400,3800,2.9988119893089404,0.0
duplication,chr22,18663101,18664600,1500,1.5262323499567148,0.0
duplication,chr22,18664801,18665900,1100,1.5074307915159397,0.0
duplication,chr22,18680801,18682700,1900,1.4498954800150479,0.0
duplicat

In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.3876702625463089,0.0
duplication,chr22,2926001,2963000,37000,3.101854428763979,0.0
duplication,chr22,4546001,4713000,167000,1.6543566565350747,0.0
duplication,chr22,6655001,6662000,7000,2.3778433283514624,0.0
duplication,chr22,8040001,8048000,8000,1.9353076895973715,0.0
duplication,chr22,9424001,9462000,38000,1.4726637389457116,0.0
duplication,chr22,13231001,13396000,165000,1.3980876760562926,0.0
duplication,chr22,13481001,13513000,32000,4.198200814322555,0.0
duplication,chr22,18705001,18881000,176000,2.7705591696600718,0.000140964195094446
duplication,chr22,19259001,19268000,9000,3.2683815660902056,0.0
duplication,chr22,20634001,20655000,21000,1.3716348016170767,0.00028363430084144843
duplication,chr22,21248001,21381000,133000,1.5138301052433971,0.00012479041184818582
duplication,chr22,21533001,21548000,15000,1.3143931131073903,0.0026

In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.6489961217564388,0.0
duplication,chr22,6530001,7010000,480000,1.0774860122497323,0.0
duplication,chr22,13230001,13320000,90000,1.3950127440530002,0.0
duplication,chr22,18710001,18880000,170000,2.7704880792151436,0.00014548537127408528
duplication,chr22,21250001,21380000,130000,1.488219271856262,0.00012480194473986933
duplication,chr22,21530001,21670000,140000,1.3415473529848065,0.00045777846649900413
duplication,chr22,24200001,24240000,40000,2.5406521314102095,4.703693444619236e-05
duplication,chr22,25150001,25310000,160000,1.2860004779039813,0.00012919104223242122
duplication,chr22,28450001,28600000,150000,2.871775641328313,4.251106414855169e-05
duplication,chr22,30800001,30980000,180000,2.9424656062077017,2.4985470948643363e-05
duplication,chr22,31150001,31360000,210000,1.4368499020967636,4.966088708533159e-05
duplication,chr22,31

In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,47100001,50100000,3000000,1.537016884563599,5.346150192325535e-05


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,46000001,50000000,4000000,1.356158761678152,5.753643565493146e-05


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app